In [ ]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

load_dotenv()

True

In [ ]:
llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    temperature=1,
    groq_api_key=os.getenv("GROQ_API_KEY")  
)
response = llm.invoke("The name of the first person to land on moon is")
print(response.content)

The name of the first person to land on the moon is Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, as part of the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.
